![Pandas logo](img/pandas.svg)

As in the prior lesson, much of this lesson is about *cleaning data*, which I have written about in more detail in [_Cleaning Data for Effective Data Science_](https://gnosis.cx/cleaning), ISBN-13 978-1801071291, 2021. I'd love it if you want to buy a copy, but you can read it freely online as well.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.training import *

# Sorting

It is often useful to look at DataFrames or Series in a particular order.  

On local disk we have an example of just one day worth of the NOAA temperature data that we utilize in various lessons.  Let us read it in then subset parts of interest.  This is still 1191 rows of data, covering each station observed that day.

In [ ]:
df = pd.read_parquet('data/NOAA-Jan1.parquet')
df

In [ ]:
df.set_index('NAME', inplace=True)
temps = df[['TEMP', 'LATITUDE', 'ELEVATION', 'PRCP', 'PRCP_ATTRIBUTES']]

In [ ]:
# Show the first 20 rows in on-disk (random-ish) order
temps.head(20)

Discussion of plots and visualization is presented in other lessons, but a nice example where order is likely to be relevant is in plotting data.  Given the random-ish order of the data, we do not see any pattern, but simply different values.

In [ ]:
# We might visualize this data, but it is mostly just a list
temps.head(20).TEMP.plot(kind='bar');

## Sort by Index

Sometimes we would like to order by the index.  In this case, where the index is simply station names, this is probably of less significance.  Scanning by alphabetical order is certainly easier than random order, but we do not expect any particular trend or pattern in the place names.

In [ ]:
# Order by the index
temps.sort_index().head(10)

## Sort by Column

Often the field we would like to order by is one of the columns, perhaps to identify patterns in how other columns behave as one field increases/decreases.

In [ ]:
# Order by temperature (coldest first)
temps.sort_values('TEMP').head()

In [ ]:
# Order by temperature (warmest first)
temps.sort_values('TEMP', ascending=False).head()

### Combine sorting and analysis

Once ordered, a more interesting pattern might emerge.

In [ ]:
# Order by temperature (coldest first)
temps.sort_values('TEMP').TEMP.head(25).plot(kind='bar');

### Sort one feature, analyze another

In fact, we need not order by the same feature we visualize or analyze.  For example, what is the pattern of temperature on January 1, ordered from highest point to lower elevations?

In [ ]:
temps.sort_values('LATITUDE', ascending=False).head(15)

In [ ]:
# Start to use Pandas "fluent style" for chained operations
(temps
     .sort_values('LATITUDE', ascending=False)
     .TEMP
     .head(30)
     .plot(kind='bar', title="Station temperature orders by latitude")
);

Just looking at the names of stations, even if they are ordered in a meaningful way, may not best reveal a general pattern among features.  We might first set the index to one feature, sort on index, then visualize or analyze another featuere so ordered.

In [ ]:
# No need to limit to small number of points if using line plot
(temps
     .set_index('LATITUDE')
     .sort_index()
     .TEMP
     .plot(title="Temperature as latitude varies")
);

We see a general pattern of temperatures tending to be higher at lower latitudes, which is generally what we would expect.  We can also see that only a few stations are above about 70° N by the fact the lines are less crowded.

Something odd seems to be happening around 55° latitude that we could take a closer look at.  For example, perhaps elevation is having a significant effect here.

In [ ]:
(temps
     .loc[(temps.LATITUDE > 55) & (temps.LATITUDE < 56)]
     .set_index('LATITUDE')
     .sort_index()
     [['TEMP', 'ELEVATION']]
     .plot(subplots=True, 
           title="Latitudes near 55°N by temperature and elevation")
);

In this particular case, no clear pattern for the oddly cold temperature at one station is evident.  We could explore further, or it may be a data anomoly.

## Sort by Multiple Columns

We can provide secondary or tertiary sort criteria when that is relevant, as well.

To make the data more manageable, let us select a smaller DataFrame of latitudes near 55° that we looked at above.

In [ ]:
temps55 = temps.loc[(temps.LATITUDE > 55) & (temps.LATITUDE < 56)].copy()
temps55

In this dataset we have a feature called `PRCP_ATTRIBUTES` which is a letter code describing the kind of precipitation.  We could consult NOAA better to understand the meanings of those codes, but for this purpose just note that there are a small number of them.

In [ ]:
temps.PRCP_ATTRIBUTES.unique()

Moreover, we would like to bin the different precipitation amounts into coarser increments just to make the secondary sort more evident.

In [ ]:
temps55 = temps.loc[(temps.LATITUDE > 55) & (temps.LATITUDE < 56)].copy()
temps55['PRCP'] = temps55.PRCP.round(1)
temps55.sort_values(['PRCP', 'PRCP_ATTRIBUTES'], 
                    ascending=[False, True],
                    inplace=True)
temps55

Unfortunately, this dataset was representing "Unknown values" as 99.99 (here rounded to 100).  So we need to ignore those placeholder values to see the meaningful part of it.

In [ ]:
temps55[temps55.PRCP < 99].head(10)

# More with Indices

The index of a Pandas DataFrame is, in a sense, just another column.  But a number of operations are much easier by utilizing an index column rather than a "regular" column.  As we have seen in prior modules, it is not difficult to move a column in or out of the index position using `.set_index()` or `.reset_index()`.

## Index alignment

Pandas performs a bit of useful magic wherein operations on several DataFrames or Series match up the rows with the corresponding index labels.  This happens regardless of the underlying order of the DataFrames, and is based on index equality not on position.

Let us look at some moderately old data files with Olympic medal counts.

In [ ]:
bronze = pd.read_csv('data/Bronze.csv', index_col='Country')[['Total']]
silver = pd.read_csv('data/Silver.csv', index_col='Country')[['Total']]
gold = pd.read_csv('data/Gold.csv', index_col=1)[['Total']]

In [ ]:
bronze.sort_values('Total', ascending=False)

We would like to find out the total number of medals won by each nation from these several DataFrames.  At first pass, we could use *index alignment* to achive this. 

In [ ]:
total = (gold + silver + bronze).sort_values('Total', ascending=False)
total.head()

So far, so good.  We could manually verify, but these top few totals are correct.  However, looking more closely, we find a problem.

In [ ]:
total.loc[['Senegal', 'Sudan', 'Kyrgyzstan']]

In [ ]:
bronze.loc[['Senegal', 'Sudan', 'Kyrgyzstan']]

In [ ]:
silver.loc[['Senegal', 'Sudan', 'Kyrgyzstan']]

### Missing values

Unfortunately for our goal, the dataset encodes the absence of medals as a `NaN` ("Not a Number").  This is a special floating point value, and is the reason that the column is not integers, which may seem more natural.  

Often in observations, some data is not available. We saw the temperature data used the placeholder value 99.99 to represent unknown precipitation.  However, there are obvious problems with that encoding: what if some location got 99.99 inches of actual rain?  

NOAA simply decided that that particular value will never occur (nor will anything more than an order-of-magnitude less).  With domain knowledge, there might be such values that are outside reasonable ranges, but they will be specific to the feature being measured.  For example, measuring precipitation in millimeters rather than inches would make 99.99 a reasonalbe and possible value.  

An additional problem with encoding missing data with placeholder values is that aggregations can go badly wrong.  For example:

In [ ]:
# What is the average precipiation across all the stations?
temps.PRCP.mean()

This conclusion is absurd.  Not only is 18.5 inches of rain not average, it is on the high side even for tropical cyclones.  Our placeholder is causing problems.  

Note that a different placeholder like -1 that was per-se numerically nonsensical would cause a similar problem, with the "average" perhaps winding up negative.

Using NaN is a more general approach.

In [ ]:
# We can use None for a missing value, Pandas stores NaN internally
temps.loc[temps.PRCP > 99, 'PRCP'] = None
temps.head()

In [ ]:
temps.PRCP.mean()

### Returning to medal count

While for a count of medals, zero is probably an excellent way of recording "No medals won," for other data fields, zero might instead simply be an actual measurement.  However, we can take our domain knowledge and impute zeros for this calulation.

In [ ]:
total = (
    (gold.fillna(0) + silver.fillna(0) + bronze.fillna(0))
    .sort_values('Total', ascending=False)
    .astype(int))
total

In [ ]:
total.loc[['Senegal', 'Sudan', 'Kyrgyzstan']]

In [ ]:
gold.loc[['Senegal', 'Sudan', 'Kyrgyzstan']]

# Exercises



Let us read in the moderately large NOAA temperature dataset we worked with before.  Our naïve manner of reading it simply allows Pandas to guess types.  For the most part that works well, but there is a date field we want to go back and adjust.

In [ ]:
url = ("https://bitbucket.org/davidmertz/sample-data/raw/"
       "61872271984f66e3094c367cf90dfc4875a22e8d/NOAA-2019-partial.csv.gz")
temperatures = pd.read_csv(url)
temperatures['DATE'] = pd.to_datetime(temperatures.DATE, format="%Y-%m-%d")
temperatures.info()

## Detecting Bad Data

Using the techniques in this lesson, as well as others you have learned, spend a while identifying all the questionable values in this large dataset.  The problem is deliberately open-ended, and a complete solution could take a long time.

For example, we have seen that precipitation uses a placeholder value.  In your cleanup, replacing placeholders with NaNs is a good improvement.  Often questionable values will fall at the `.head()` or the `.tail()` of the large DataFrame once you sort appropriately.

In the below we see that there are many 0.00 precipitation amounts, that are probably genuine measures.  There are a large number if placeholder values of 99.99.  Then there are some number of non-zero precipitation amounts.  Try to quantify that further. 

In [ ]:
# Look at start of sorted data, start, middle, jump from end, end
nrows = len(temperatures)
(temperatures
     .sort_values('PRCP')
     .iloc[[0, nrows//2, -70_000, -1]]
     .loc[:, ['NAME', 'DATE', 'TEMP', 'PRCP']]
     .set_index('NAME')
)

In [ ]:
# Identify placeholders
...

## Outliers

Of course, sometimes unusual measurements occur and reflect unusual events.  But bad data is also common in large datasets (even in small datasets).  Try to identify questionable measurements that are not actually placeholder values.

For example, in our analysis in this module, we found that one temperature measurement on January 1 was dramatically lower than other values at the same 55° latitude.  This is far enough off to be likely to be a data error of some kind.

In [ ]:
print("Number of measurements:", len(temps55))
print("Lowest temperature (F):", temps55.TEMP.min())
print("Highest temperature:   ", temps55.TEMP.max())
print("Mean temp Jan 1:       ", temps55.TEMP.mean())

Try to identify similarly questionable data among the other 425k rows, or simply to verify whether this datum is actually an error (e.g. we did not explain it by a high elevation, but perhaps there is another reason we missed). 

In [ ]:
# Identify outliers
...

*Suggestion*: data that differs by 3 standard deviations from "similar data" is likely to be an error.  For example, if a measurement one day, at one station, is dramatically different from the days nearby at same station, something is suspicious.  Or likewise, if data is very uncharacteristic of its month, latitude, and/or elevation, that could be a data error.

Of course, there really *are* hot days and cold days at the same place. And windy days and gentle days. And rainy days and clear days.  That is why the dataset exists to start with.  This exercise requires judgement.

## Merging Subsets

We have seen techniques to subset DataFrames, as well as to perform reductions.  Using those techniques, create a report on the following.  For each of the 1100 stations, what was the difference between the average January temperature and the average February temperature?

*Note:* In later modules we will explore other ways to answer this question and similar ones.  You have sufficient tools now to do it already though.

In [ ]:
# Identify tendencies between January and February
...

If you have extra time, formulate similar answer for other time period comparisons and other measured features.